In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader
from torch.jit import RecursiveScriptModule

# other libraries
from typing import Final

# own modules
from src.RNNModelTrain.data import load_data
from src.RNNModelTrain.utils import set_seed
#from src.RNNModelTrain.train_functions import t_step


from lime.lime_text import LimeTextExplainer

%matplotlib inline

In [6]:
# static variables
DATA_PATH: Final[str] = "NLP_Data/data"
NUM_CLASSES: Final[int] = 10

# set device
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)

In [3]:
# load the model
model: RecursiveScriptModule = torch.jit.load("models/model.pt")

In [8]:
# load the data
train_dataloader: DataLoader
val_dataloader: DataLoader
test_dataloader: DataLoader
mean_price: float
std_price: float
(
    train_dataloader, val_dataloader, test_dataloader
) = load_data(save_path=DATA_PATH)

c:\Users\pablo\Desktop\IMAT\Tercero\NaturalLanguageProcessing\rnn_explainability\src\RNNModelTrain\data.py:62: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['tag'] = data['tag'].replace('bot', 1)
c:\Users\pablo\Desktop\IMAT\Tercero\NaturalLanguageProcessing\rnn_explainability\src\RNNModelTrain\data.py:62: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['tag'] = data['tag'].replace('bot', 1)
c:\Users\pablo\Desktop\IMAT\Tercero\NaturalLanguageProcessing\rnn_explainability\src\RNNModelTrain\data.py:62: FutureWarning: Downcast

In [18]:
# Print some example data
for x, y, _ in train_dataloader:
    print("Shape of x [batch_size, seq_length, input_dim]:", x.shape)
    print("Shape of y [batch_size]:", y.shape)
    
    print(x[0])
    break
    
    

Shape of x [batch_size, seq_length, input_dim]: torch.Size([64, 41])
Shape of y [batch_size]: torch.Size([64])
tensor([  20,   47,  315,   27,  464, 4153, 1075,   20,   47,  159,   58,   43,
         153,   20,   47,   13,  315,  528, 4153, 1075, 1792,  159,   43, 1841,
           8,   20,   47,   13,   58,   38,   47,   13,   20,   23,  888,   96,
           3,    4,   13,   11,  132])


In [ ]:
"""# evaluate the model
test_results = t_step(model, test_dataloader,
                        mean_price, std_price, device)

# print the test resulsts
print(f"MAE test loss: {test_results}")"""

Explain the model with LIME:

In [14]:
X_train = np.array([])
y_train = np.array([])
for x, y, _ in train_dataloader:
    X_train = np.append(X_train, x)
    y_train = np.append(y_train, y)
    
X_test = np.array([])
y_test = np.array([])
for x, y, _ in test_dataloader:
    X_test = np.append(X_test, x)
    y_test = np.append(y_test, y)
    
print(X_test[0])

20.0


In [13]:
explainer = lime_tabular.RecurrentTabularExplainer(X_train, training_labels=y_train, feature_names=data_columns,
                                                   discretize_continuous=True,
                                                   class_names=['Falling', 'Rising'],
                                                   discretizer='decile')

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
exp = explainer.explain_instance(X_test[50], model.predict, num_features=10, labels=(1,))
exp.show_in_notebook()